In [1]:
import pandas as pd
import numpy as np
import re
from typing import List, Dict, Optional

In [13]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="ProsusAI/finbert")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


In [2]:
df = pd.read_csv("/content/drive/MyDrive/esg_data/clean_news.csv")

In [3]:
df

,symbol,url,text_content,cleaned_text,text_content_word_count,cleaned_text_word_count
0,FAST,https://www.wallstreet-online.de/nachricht/186...,Fastenal Company (Nasdaq: FAST) (herein 'Faste...,Fastenal Company Nasdaq FAST herein 'Fastenal'...,210,209
1,SMAWF,https://www.pandct.com/news/siemens-achieves-s...,Siemens achieves strong progress on sustainabi...,Siemens achieves strong progress on sustainabi...,1368,1359
2,SMAWF,https://www.webwire.com/ViewPressRel.asp?aId=3...,Decarbonization program with UAEs Ministry of...,Decarbonization program with UAEs Ministry of ...,700,697
3,SMAWF,https://www.prnewswire.com/news-releases/autom...,"NEW YORK, Oct. 14, 2024 /PRNewswire/ -- Report...","NEW YORK, Oct. 14, 2024 PRNewswire Report on h...",1738,1730
4,SMAWF,https://menafn.com/1108777919/Automation-Solut...,"(MENAFN- PR Newswire) NEW YORK, Oct. 14, 2024 ...","MENAFN PR Newswire NEW YORK, Oct. 14, 2024 PRN...",1755,1748
...,...,...,...,...,...,...
2372,8411.T,https://www.thehindubusinessline.com/companies...,"16:47 | December 13, 2024\n\nThe Group of Mini...","1647 December 13, 2024 The Group of Ministers ...",199,198
2373,8411.T,https://www.thehindubusinessline.com/companies...,"16:47 | December 13, 2024\n\nThe Group of Mini...","1647 December 13, 2024 The Group of Ministers ...",199,198
2374,8411.T,https://energycentral.com/c/ec/doe-awards-six-...,DOE Awards Six HALEU Fuel Contracts\n\nUrenco ...,DOE Awards Six HALEU Fuel Contracts Urenco Boo...,4242,4200
2375,8411.T,https://www.thehindubusinessline.com/companies...,"16:47 | December 13, 2024\n\nThe Group of Mini...","1647 December 13, 2024 The Group of Ministers ...",199,198


In [4]:
agg = df.groupby('symbol')['cleaned_text'].agg(list).reset_index()

In [5]:
agg.shape

(399, 2)

In [6]:
agg

,symbol,cleaned_text
0,000270.KS,[The Integrated Greenhouse Gas Information Sys...
1,0005.HK,[We recently published a list of 12 Best ADR S...
2,000660.KS,"[According to a new report from SP Global, the..."
3,005380.KS,"[The series, now viewable on YouTube, presents..."
4,005490.KS,"[0858 October 22, 2024 New Delhi, October 22, ..."
...,...,...
394,YESBANK.BO,"[0858 October 22, 2024 New Delhi, October 22, ..."
395,YRAIF,[Yara has transferred to Fernanda Lopes Larsen...
396,YUM,[Yum! Brands Continues to Make Progress on Car...
397,ZTS.MX,"[In October, 2022, The Globe and Mails Jeffrey..."


In [14]:
sample_articles = pipe(agg['cleaned_text'][0], padding=True, truncation=True)
sample_articles

[{'label': 'positive', 'score': 0.7086213231086731},
 {'label': 'neutral', 'score': 0.5014052987098694},
 {'label': 'negative', 'score': 0.45283085107803345},
 {'label': 'negative', 'score': 0.5059974789619446},
 {'label': 'neutral', 'score': 0.5762618780136108},
 {'label': 'neutral', 'score': 0.8346115350723267},
 {'label': 'neutral', 'score': 0.4963551163673401}]

In [12]:
def calculate_sentiment_aggregates(articles: List[str]):
  sample_articles = pipe(articles, padding=True, truncation=True)
  return sample_articles

In [14]:
agg.head()

,symbol,cleaned_text
0,000270.KS,[The Integrated Greenhouse Gas Information Sys...
1,0005.HK,[We recently published a list of 12 Best ADR S...
2,000660.KS,"[According to a new report from SP Global, the..."
3,005380.KS,"[The series, now viewable on YouTube, presents..."
4,005490.KS,"[0858 October 22, 2024 New Delhi, October 22, ..."


In [15]:
agg['raw_sentiment'] = agg.apply(lambda x: calculate_sentiment_aggregates(x['cleaned_text']), axis=1)

In [16]:
agg

,symbol,cleaned_text,raw_sentiment
0,000270.KS,[The Integrated Greenhouse Gas Information Sys...,"[{'label': 'positive', 'score': 0.708621323108..."
1,0005.HK,[We recently published a list of 12 Best ADR S...,"[{'label': 'negative', 'score': 0.481453657150..."
2,000660.KS,"[According to a new report from SP Global, the...","[{'label': 'positive', 'score': 0.569136857986..."
3,005380.KS,"[The series, now viewable on YouTube, presents...","[{'label': 'neutral', 'score': 0.7179769873619..."
4,005490.KS,"[0858 October 22, 2024 New Delhi, October 22, ...","[{'label': 'neutral', 'score': 0.5735214352607..."
...,...,...,...
394,YESBANK.BO,"[0858 October 22, 2024 New Delhi, October 22, ...","[{'label': 'neutral', 'score': 0.5735214352607..."
395,YRAIF,[Yara has transferred to Fernanda Lopes Larsen...,"[{'label': 'neutral', 'score': 0.9550024867057..."
396,YUM,[Yum! Brands Continues to Make Progress on Car...,"[{'label': 'positive', 'score': 0.742373228073..."
397,ZTS.MX,"[In October, 2022, The Globe and Mails Jeffrey...","[{'label': 'neutral', 'score': 0.8865858316421..."


In [17]:
def direct_count_ignore_neutral(articles: List[Dict[str, float]]) -> Optional[float]:
    """
    Computes the sentiment index by counting positive and negative articles,
    ignoring neutrals in the numerator but including them in the total count.

    Formula:
      S = (Number of positive articles - Number of negative articles) / Total articles

    Args:
        articles: List of dictionaries with keys 'label' (expected to be 'positive', 'negative', or 'neutral')
                  and 'score' (confidence, not used in this count-based method).

    Returns:
        The sentiment index (float) or None if the article list is empty.
    """
    if not articles:
        return None

    total = len(articles)
    pos_count = sum(1 for article in articles if article.get('label') == 'positive')
    neg_count = sum(1 for article in articles if article.get('label') == 'negative')

    sentiment_index = (pos_count - neg_count) / total
    return sentiment_index

In [18]:
def direct_count_extremes(articles: List[Dict[str, float]]) -> Optional[float]:
    """
    Computes the sentiment index considering only articles with positive or negative sentiments.

    Formula:
      S = (Number of positive articles - Number of negative articles) / (Number of positive articles + Number of negative articles)

    Args:
        articles: List of dictionaries with keys 'label' and 'score'.

    Returns:
        The sentiment index (float) or None if there are no extreme articles.
    """
    pos_count = sum(1 for article in articles if article.get('label') == 'positive')
    neg_count = sum(1 for article in articles if article.get('label') == 'negative')

    if pos_count + neg_count == 0:
        return None  # or return 0 if you prefer to indicate neutrality when no extremes exist

    sentiment_index = (pos_count - neg_count) / (pos_count + neg_count)
    return sentiment_index

In [19]:
def weighted_sentiment_normal_by_article(articles: List[Dict[str, float]]) -> Optional[float]:
    """
    Computes the weighted sentiment index normalized by the total number of articles.

    Each article's sentiment score is mapped as:
      - positive: +1
      - neutral: 0
      - negative: -1

    Formula:
      S = (1 / N) * Σ (confidence * sentiment score) over all articles

    Args:
        articles: List of dictionaries with keys 'label' and 'score' (confidence).

    Returns:
        The weighted sentiment score (float) or None if the articles list is empty.
    """
    if not articles:
        return None

    # Mapping from label to sentiment score.
    label_to_score = {'positive': 1, 'neutral': 0, 'negative': -1}
    total_articles = len(articles)

    weighted_sum = 0.0
    for article in articles:
        label = article.get('label')
        confidence = article.get('score', 0)
        sentiment_value = label_to_score.get(label, 0)
        weighted_sum += confidence * sentiment_value

    sentiment_index = weighted_sum / total_articles
    return sentiment_index


In [20]:
def weighted_sentiment_normal_by_confidence(articles: List[Dict[str, float]]) -> Optional[float]:
    """
    Computes the weighted sentiment index normalized by the sum of confidence scores.

    Uses the same sentiment mapping as the other weighted function:
      - positive: +1
      - neutral: 0
      - negative: -1

    Formula:
      S = Σ (confidence * sentiment score) / Σ (confidence)

    Args:
        articles: List of dictionaries with keys 'label' and 'score' (confidence).

    Returns:
        The weighted sentiment score (float) or None if the total confidence is zero.
    """
    # Mapping from label to sentiment score.
    label_to_score = {'positive': 1, 'neutral': 0, 'negative': -1}

    weighted_sum = 0.0
    total_confidence = 0.0
    for article in articles:
        label = article.get('label')
        confidence = article.get('score', 0)
        sentiment_value = label_to_score.get(label, 0)
        weighted_sum += confidence * sentiment_value
        total_confidence += confidence

    if total_confidence == 0:
        return None

    sentiment_index = weighted_sum / total_confidence
    return sentiment_index

In [15]:
print("Direct Count (Ignoring neutrals):", direct_count_ignore_neutral(sample_articles))
print("Direct Count (Only extremes):", direct_count_extremes(sample_articles))
print("Weighted Sentiment (Normalized by articles):", weighted_sentiment_normal_by_article(sample_articles))
print("Weighted Sentiment (Normalized by confidence):", weighted_sentiment_normal_by_confidence(sample_articles))

Direct Count (Ignoring neutrals): -0.14285714285714285
Direct Count (Only extremes): -0.3333333333333333
Weighted Sentiment (Normalized by articles): -0.03574385813304356
Weighted Sentiment (Normalized by confidence): -0.06138417136902733


In [23]:
agg['sentiment_ratio_all'] = agg.apply(lambda x: direct_count_ignore_neutral(x['raw_sentiment']), axis=1)
agg['sentiment_ratio_extremes'] = agg.apply(lambda x: direct_count_extremes(x['raw_sentiment']), axis=1)
agg['weighted_sentiment_avg'] = agg.apply(lambda x: weighted_sentiment_normal_by_article(x['raw_sentiment']), axis=1)
agg['weighted_sentiment_norm_conf'] = agg.apply(lambda x: weighted_sentiment_normal_by_confidence(x['raw_sentiment']), axis=1)

In [24]:
agg

,symbol,cleaned_text,raw_sentiment,sentiment_ratio_all,sentiment_ratio_extremes,weighted_sentiment_avg,weighted_sentiment_norm_conf
0,000270.KS,[The Integrated Greenhouse Gas Information Sys...,"[{'label': 'positive', 'score': 0.708621323108...",-0.142857,-0.333333,-0.035744,-0.061384
1,0005.HK,[We recently published a list of 12 Best ADR S...,"[{'label': 'negative', 'score': 0.481453657150...",-0.500000,-1.000000,-0.317086,-0.410726
2,000660.KS,"[According to a new report from SP Global, the...","[{'label': 'positive', 'score': 0.569136857986...",0.428571,1.000000,0.256201,0.386590
3,005380.KS,"[The series, now viewable on YouTube, presents...","[{'label': 'neutral', 'score': 0.7179769873619...",0.166667,1.000000,0.122389,0.174893
4,005490.KS,"[0858 October 22, 2024 New Delhi, October 22, ...","[{'label': 'neutral', 'score': 0.5735214352607...",0.000000,NaN,0.000000,0.000000
...,...,...,...,...,...,...,...
394,YESBANK.BO,"[0858 October 22, 2024 New Delhi, October 22, ...","[{'label': 'neutral', 'score': 0.5735214352607...",0.000000,NaN,0.000000,0.000000
395,YRAIF,[Yara has transferred to Fernanda Lopes Larsen...,"[{'label': 'neutral', 'score': 0.9550024867057...",0.200000,1.000000,0.111325,0.135677
396,YUM,[Yum! Brands Continues to Make Progress on Car...,"[{'label': 'positive', 'score': 0.742373228073...",0.200000,1.000000,0.148475,0.197189
397,ZTS.MX,"[In October, 2022, The Globe and Mails Jeffrey...","[{'label': 'neutral', 'score': 0.8865858316421...",0.000000,NaN,0.000000,0.000000


In [27]:
agg.to_csv("/content/drive/MyDrive/esg_data/news_sentiment.csv", index=False)